<a href="https://colab.research.google.com/github/datacommonsorg/api-python/blob/master/notebooks/intro_data_science/Regression_Evaluation_and_Interpretation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Copyright 2022 Google LLC.
SPDX-License-Identifier: Apache-2.0

# Regression: Evaluation and Interpretation
In part 1, we saw how powerful regression can be as a tool for prediction. In this colab, we'll take that exploration one step further: what can regression models tell us about the statistical relationships between variables?

In particular, this colab will take a more rigorous statistical approach to regressions. We'll look at how to evaluate and interpret our regression models using statistical methods.

## Learning Objectives:
* Hypothesis testing with regression
* Regression tables
* Pearson Correlation coefficient, $r$
* $R^2$ and adjusted $R^2$
* Interpreting weights and intercepts
* How correlated variables affect models
---
**Need extra help?**

If you're new to Google Colab, take a look at [this getting started tutorial](https://colab.research.google.com/notebooks/intro.ipynb).

To build more familiarity with the Data Commons API, check out these [Data Commons Tutorials](https://docs.datacommons.org/tutorials/).

And for help with Pandas and manipulating data frames, take a look at the [Pandas Documentation](https://pandas.pydata.org/docs/reference/index.html).

We'll be using the scikit-learn library for implementing our models today. Documentation can be found [here](https://scikit-learn.org/stable/modules/classes.html). 

As usual, if you have any other questions, please reach out to your course staff!

# Getting Set Up


Run the following code boxes to load the python libraries and data we'll be using today.

In [ ]:
# Setup/Imports
!pip install datacommons --upgrade --quiet
!pip install datacommons_pandas --upgrade --quiet

In [ ]:
# Data Commons Python and Pandas APIs
import datacommons
import datacommons_pandas

# For manipulating data
import numpy as np
import pandas as pd

# For implementing models and evaluation methods
from sklearn import linear_model
from sklearn.metrics import r2_score, mean_squared_error
from statsmodels import api as sm


# For plotting/printing
from matplotlib import pyplot as plt
import seaborn as sns

## The Data

In this assignment, we'll be returning to the scenario we started in Part 1. As a refresher, we'll be exploring how obesity rates vary with different health or societal factors across US cities.

Our data science question: **What can we learn about the relationship of those health and lifestyle factors to obesity rates?**

In [ ]:
# Load the data we'll be using
city_dcids = datacommons.get_property_values(["CDC500_City"],
                                             "member",
                                             limit=500)["CDC500_City"]

# We've compiled a list of some nice Data Commons Statistical Variables
# to use as features for you
stat_vars_to_query = [
  "Count_Person",
  "Percent_Person_PhysicalInactivity",
  "Percent_Person_SleepLessThan7Hours",
  "Percent_Person_WithHighBloodPressure",
  "Percent_Person_WithMentalHealthNotGood",
  "Percent_Person_WithHighCholesterol",
  "Percent_Person_Obesity"
                      
]

# Query Data Commons for the data and remove any NaN values
raw_features_df = datacommons_pandas.build_multivariate_dataframe(city_dcids,stat_vars_to_query)
raw_features_df.dropna(inplace=True)

# order columns alphabetically
raw_features_df = raw_features_df.reindex(sorted(raw_features_df.columns), axis=1)

# Add city name as a column for readability.
# --- First, we'll get the "name" property of each dcid
# --- Then add the returned dictionary to our data frame as a new column
df = raw_features_df.copy(deep=True)
city_name_dict = datacommons.get_property_values(city_dcids, 'name')
city_name_dict = {key:value[0] for key, value in city_name_dict.items()}
df.insert(0, 'City Name', pd.Series(city_name_dict))

# Display results
display(df)

,City Name,Count_Person,Percent_Person_Obesity,Percent_Person_PhysicalInactivity,Percent_Person_SleepLessThan7Hours,Percent_Person_WithHighBloodPressure,Percent_Person_WithHighCholesterol,Percent_Person_WithMentalHealthNotGood
place,,,,,,,,
geoId/0107000,Birmingham,200733,40.1,33.9,44.0,44.9,34.6,17.4
geoId/0135896,Hoover,92606,28.8,20.2,33.4,32.4,33.2,12.0
geoId/0137000,Huntsville,215006,34.7,28.4,38.7,37.4,33.5,15.4
geoId/0150000,Mobile,187041,38.0,29.9,41.0,42.2,35.2,16.3
geoId/0151000,Montgomery,200603,36.7,30.7,42.6,41.2,34.5,17.0
...,...,...,...,...,...,...,...,...
geoId/5548000,Madison,269840,24.9,17.1,31.2,22.1,28.2,13.0
geoId/5553000,Milwaukee,577222,38.9,28.9,38.8,31.2,29.9,16.6
geoId/5566000,Racine,77816,43.2,28.8,36.7,31.6,33.1,15.1


## The Model

Run the following code box to fit an [ordinary least squares](https://en.wikipedia.org/wiki/Ordinary_least_squares) regression model to our data.

In [ ]:
# fit a regression model
dep_var = "Percent_Person_Obesity"
y = df[dep_var].to_numpy().reshape(-1, 1)
x = df.loc[:, ~df.columns.isin([dep_var, "City Name"])]
x = sm.add_constant(x)


model = sm.OLS(y, x)
results = model.fit()

# Part 0) Regression Tables

When performing regression analyses, statistical packages will usually provide a _**regression table**_, which summarizes the results of the analysis.

Run the following codebox to display the regression table for our original model. In this colab, we'll go over some of the statistics included in the table.


In [ ]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.715
Model:                            OLS   Adj. R-squared:                  0.711
Method:                 Least Squares   F-statistic:                     205.7
Date:                Tue, 11 Jan 2022   Prob (F-statistic):          1.29e-130
Time:                        23:46:37   Log-Likelihood:                -1283.7
No. Observations:                 499   AIC:                             2581.
Df Residuals:                     492   BIC:                             2611.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                                             coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------

# Part 1) Hypothesis Testing


## 1.1) Null Hypotheses

When performing statistical analyses, one usually starts with a statement of the null hypothesis. Typically for regression models, these take the form of the coefficient for a variable equaling zero.

**Q1.2)** Write out the null hypotheses for each of our independent variables.

## 1.2) T-test

So how do we test our null hypotheses? We use the [T-test](https://en.wikipedia.org/wiki/Student%27s_t-test#Slope_of_a_regression_line).

Take a look at the regression table above to answer the following questions

**Q1.2A)** According to the t-test, which variables are statistically significant?

**Q1.2B)** For variables that are not statistically significant, should we keep them in our model? Why or why not?

## 1.3) F-test

Beyond testing the significance of our individual variables independently, we can also test the significance of our model overall using the [F-test](https://en.wikipedia.org/wiki/F-test#Regression_problems). In particular, the F-test compares our model to one without predictors (aka, just an intercept). In other words, can our model do statistically better than just predicting the mean?

Again use the regression table above to answer the following questions:

**Q1.3A)** What is the null hypothesis for the F-test?

**Q1.3B)** Can we reject the null hypothesis for our model?

# Part 2) Statistical Measures

## 2.1) Correlation Coefficient $r$

We can quantify predictiveness of variables using a _correlation coefficient_, a number that represents the degree to which two variables have a statistical relationship. The most common correlation coefficient used is the [Pearson correlation coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient), also known as _Pearson's r_, which measures the strength of linear relationships between variables.

Mathematically, the correlation coefficient is defined as:
$$ r = \frac{\sum_i (x_i - \bar{x})(y_i - \bar{y})}{\sqrt{\sum_i (x_i - \bar{x})^2}\sqrt{\sum_i (y_i - \bar{y})^2}}
$$

where $x$ and $y$ are the two variables.

Those of you with a statistics background might recognize this as the ratio of covariance to the product of their standard deviations.

**Q2.1A)** Either using the mathematical definition or by exploring with code, explain what the correlation coefficient would be in the following cases:

A) $x = y$

B) $x = -y$

C) $x$ and $y$ are both normally distributed variables with mean 0 and variance 1, randomly sampled independently from each other.

In [ ]:
"""
Optional cell for Q2.1A
"""

# Hint: Try writing code to generate values for x and y, then either write or import
# a function to calculate the correlation coefficient

# Your code here

Now run the following code box to use panda's `.corr()`  function to calculate the correlation coefficient between our variables. Note that pandas outputs the results as a matrix.

In [ ]:
# calculate correlation
df.corr()

,Count_Person,Percent_Person_Obesity,Percent_Person_PhysicalInactivity,Percent_Person_SleepLessThan7Hours,Percent_Person_WithHighBloodPressure,Percent_Person_WithHighCholesterol,Percent_Person_WithMentalHealthNotGood
Count_Person,1.000000,-0.020728,0.041395,0.083870,-0.006448,-0.027940,-0.011120
Percent_Person_Obesity,-0.020728,1.000000,0.797189,0.651754,0.719570,0.428489,0.719002
Percent_Person_PhysicalInactivity,0.041395,0.797189,1.000000,0.763090,0.750962,0.505813,0.740931
Percent_Person_SleepLessThan7Hours,0.083870,0.651754,0.763090,1.000000,0.696953,0.380698,0.675784
Percent_Person_WithHighBloodPressure,-0.006448,0.719570,0.750962,0.696953,1.000000,0.751325,0.545973
Percent_Person_WithHighCholesterol,-0.027940,0.428489,0.505813,0.380698,0.751325,1.000000,0.233295
Percent_Person_WithMentalHealthNotGood,-0.011120,0.719002,0.740931,0.675784,0.545973,0.233295,1.000000



**Q2.1B)** Explain why the diagonals of the matrix have the value 1.

**Q2.1C)** What is the correlation coefficient between `Count_Person` and `Percent_Person_Obesity`? What does the correlation coefficient imply about the relationship between population and obesity rate?

**Q2.1D)** What is the correlation coefficient between `Percent_Person_PhysicalInactivity` and `Percent_Person_Obesity`? What does the correlation coefficient imply about the relationship between physical inactivity and obesity rate?

**Q2.1E)** In general, would you prefer to include features that correlate strongly with the dependent variable, or features with no correlation in a regression model?

**Q2.1F)** You find a new feature with correlation coefficient $r=-0.97$ between it and obesity rates. Would it be a good idea to add this new feature to your model?


## 2.2) $R^2$ Score

To quantify how predictive a linear regression model is overall is to use the [coefficient of determination](https://en.wikipedia.org/wiki/Coefficient_of_determination), $R^2$ (pronounced "R squared").

Mathematically, the $R^2$ score is defined as:

$$S_{residuals} = \sum_i{(y_i - f_i)^2} \\
S_{total} = \sum_i{(y_i - \bar{y})^2}\\
R^2 = 1 - \frac{S_{residuals}}{S_{total}}$$

where $y_i$s are the actual dependent variable values, $f_i$ are the predicted dependent variable values, and $\bar{y}$ is the average of the $y_i$'s.

Conceptually, the $R^2$ score is a measure of explained variance. If $R^2=0.75$, that means that 75% of the variance in the dependent variable has been accounted for by our model, while 25% of the remaining variability has not.

**Q2.2A)** Based on the mathematic definition, what is the range of values possible for R^2?

**Q2.2B)** Come up with a situation (e.g. what would the data look like) where:

A) $R^2 = 1.0$

B) $R^2 = 0.0$

Let's now analyze what the $R^2$ value is for our model.

In [ ]:
# calculate R^2
print("Model R^2 =", results.rsquared)

Model R^2 = 0.7149364375531281


**Q2.2C)** Is the model's $R^2$ a "good" score?

**Q2.2D)** Can you think of any ways we can change our model that would improve the $R^2$ score?

## 2.3) Adjusted $R^2$

There's an issue with $R^2$ scores that one needs to be aware of when working with multiple independent variables. Namely, that the number of independent variables used can affect the $R^2$ score.

Let's see this in practice. Let's create a new dataframe with an extra 100 dummy variables (randomly sampled from a 0-mean 1-variance normal distribution) tacked on.

In [ ]:
# Pad our dataframe with more random variables
df_padded = df.copy()
num_rows = len(df.index)
for i in range(100):
  var_name = f"Random Variable {i}"
  random_data = np.random.normal(size=(num_rows, 1))
  df_padded[var_name] = random_data
display(df_padded)


,City Name,Count_Person,Percent_Person_Obesity,Percent_Person_PhysicalInactivity,Percent_Person_SleepLessThan7Hours,Percent_Person_WithHighBloodPressure,Percent_Person_WithHighCholesterol,Percent_Person_WithMentalHealthNotGood,Random Variable 0,Random Variable 1,Random Variable 2,Random Variable 3,Random Variable 4,Random Variable 5,Random Variable 6,Random Variable 7,Random Variable 8,Random Variable 9,Random Variable 10,Random Variable 11,Random Variable 12,Random Variable 13,Random Variable 14,Random Variable 15,Random Variable 16,Random Variable 17,Random Variable 18,Random Variable 19,Random Variable 20,Random Variable 21,Random Variable 22,Random Variable 23,Random Variable 24,Random Variable 25,Random Variable 26,Random Variable 27,Random Variable 28,Random Variable 29,Random Variable 30,Random Variable 31,...,Random Variable 60,Random Variable 61,Random Variable 62,Random Variable 63,Random Variable 64,Random Variable 65,Random Variable 66,Random Variable 67,Random Variable 68,Random Variable 69,Random Variable 70,Random Variable 71,Random Variable 72,Random Variable 73,Random Variable 74,Random Variable 75,Random Variable 76,Random Variable 77,Random Variable 78,Random Variable 79,Random Variable 80,Random Variable 81,Random Variable 82,Random Variable 83,Random Variable 84,Random Variable 85,Random Variable 86,Random Variable 87,Random Variable 88,Random Variable 89,Random Variable 90,Random Variable 91,Random Variable 92,Random Variable 93,Random Variable 94,Random Variable 95,Random Variable 96,Random Variable 97,Random Variable 98,Random Variable 99
place,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
geoId/0107000,Birmingham,200733,40.1,33.9,44.0,44.9,34.6,17.4,-1.775024,1.448878,0.751420,-0.337571,-1.630007,-0.791610,-0.764136,1.001150,-0.497411,1.137086,0.388179,1.490722,0.479426,0.885647,2.139369,0.980127,-0.024894,-0.594549,0.057923,-0.206539,-0.676724,-2.132997,-0.659994,-1.009111,-0.611383,0.865748,-1.102444,0.305733,1.471427,-1.673124,-0.362989,0.163016,...,-1.721955,0.007987,-1.713295,-0.960122,0.449155,1.645943,-1.917358,0.040276,0.664790,-0.296875,0.954265,-1.586074,-1.319715,-1.551457,-0.404962,-0.581372,1.094329,-1.063519,1.249983,0.010912,-0.910707,0.438521,-0.202320,1.334286,-0.084193,0.414590,-0.209882,-1.603098,0.403013,-0.350851,-0.904914,1.062353,-0.370768,-0.365375,0.908413,-0.311293,0.084229,-0.442015,0.031837,-0.767270
geoId/0135896,Hoover,92606,28.8,20.2,33.4,32.4,33.2,12.0,0.694015,1.150745,0.025378,-0.283279,-1.503832,0.690609,-1.694068,0.082391,0.749624,-0.887541,-0.499350,-0.361166,0.946467,1.292919,-0.237458,2.216770,-1.168924,0.115793,0.216861,-1.016504,0.238576,-1.032536,0.744089,-0.619010,-0.562913,0.367186,0.847115,1.787353,0.734604,-0.638914,-0.946534,-0.530487,...,-0.979663,-0.370279,-1.037029,1.415770,1.118652,-0.677876,1.302612,-1.908806,1.756797,1.210258,1.005200,0.789094,-1.573405,2.561985,0.498816,-1.946839,-0.120957,-0.996368,-0.184924,-1.579735,0.392238,0.430946,-0.416738,1.785403,-2.073653,0.066519,1.128013,-0.126520,-0.112078,0.911456,-1.264666,-0.370690,1.162111,-0.785190,0.546058,0.844673,-0.371177,-1.715979,0.975173,-0.480216
geoId/0137000,Huntsville,215006,34.7,28.4,38.7,37.4,33.5,15.4,0.541813,-0.882819,0.958483,-0.261622,-0.318740,-0.296085,1.120216,0.625789,-0.255850,-0.295485,0.135865,0.272344,1.187588,-1.112298,0.068120,0.952634,0.141764,-0.210001,-0.478465,0.546696,-0.439181,0.087348,-1.092643,-0.371697,2.281920,-1.947865,-0.327443,0.040106,0.839419,0.002466,-0.918838,1.645719,...,1.755844,0.440961,-0.859049,-1.748428,0.187462,-0.103037,1.659586,-0.516001,-0.926824,1.735668,1.421723,-0.243201,-0.173084,0.046470,-0.996634,-0.168138,-0.195259,1.266686,-1.125591,0.749213,-0.993767,0.052149,0.014974,-0.610232,1.284871,1.321688,2.192499,-1.349410,0.056769,0.868246,0.590667,0.343471,-0.236944,-0.183300,-0.755913,0.290493,-1.092547,0.499298,0.546659,-1.824621
geoId/0150000,Mobile,187041,38.0,29.9,41.0,42.2,35.2,16.3,-1.411763,0.970479,

Now let's fit a new model to the data and compare R^2 scores.

In [ ]:
# New R^2
y_padded = df_padded[dep_var].to_numpy().reshape(-1, 1)
x_padded = df_padded.loc[:, ~df_padded.columns.isin([dep_var, "City Name"])]
x_padded = sm.add_constant(x_padded)

padded_model = sm.OLS(y_padded, x_padded)
padded_results = padded_model.fit()

print("Original Model R^2 = ", results.rsquared)
print("Padded Model R^2 =", padded_results.rsquared)


Original Model R^2 =  0.7149364375531281
Padded Model R^2 = 0.769737804849763


**Q2.3A)** Which model had a better $R^2$ score?

**Q2.3B)** Think about the variables used in each model. Should one model be much more predictive than another?

**Q2.3B)** In general, how would you expect $R^2$ to change as we increase the number of independent variables?



So how do we fix this? We can adjust our $R^2$ metric to account for the number of variables. The most popular way to defined the _**adjusted $R^2$**_ score is as follows:

$$R^{2}_{adj}=1-(1-R^{2}){n-1 \over n-p-1}$$

where $n$ is the number of data points and $p$ is the number of independent variables.

Now let's compare the adjusted $R^2$ of our models.

In [ ]:
# Adjusted R^2
print("Original Model Adjusted R^2 = ", results.rsquared_adj)
print("Padded Model Adjusted R^2 =", padded_results.rsquared_adj)

Original Model Adjusted R^2 =  0.7114600526452394
Padded Model Adjusted R^2 = 0.7074730275897498


**Q2.3D)** Which model had a better adjusted $R^2$ score?

**Q2.3E)** When would you prefer to use adjusted R^2 over R^2 to evaluate model fit?

# Part 3) Interpreting Regression Models


## 3.1) Analyzing Weights and Intercepts
The parameters of the regression model itself can also yield important insights.

Run the following code box to display the weights and intercept of our original model.

In [ ]:
# Display weights/coefficients
display(results.params.round(5))

const                                     9.04913
Count_Person                             -0.00000
Percent_Person_PhysicalInactivity         0.41560
Percent_Person_SleepLessThan7Hours       -0.12346
Percent_Person_WithHighBloodPressure      0.47605
Percent_Person_WithHighCholesterol       -0.25152
Percent_Person_WithMentalHealthNotGood    0.70090
dtype: float64

**Q3.1A)** What is the intercept of our model? What are its units?

**Q3.1B)** What are the units on each of the model weights (aka coefficients)?

**Q3.1C)** Which variables matter most to our model?

**Q3.1D)** In words, describe what a weight/coefficient in a linear regression means.

**Q3.1E)** Our model is used to generate a predicted obesity rate for a fictional city named Dataopolis. If we increased `Percent_Person_WithMentalHealthNotGood` for Dataopolis by 1 unit, _while keeping the values for all remaining variables constant_, by how much would we expect our predicted obesity rate to change?

## 3.2) The effect of correlated variables

When interpreting weights, one thing to look out for is if we have independent variables that are highly correlated with each other.

Let's illustrate why this might be a problem, by adding a variables that are correlated with one of the existing variables

In [ ]:
# New variable correlated with Percent_Person_WithMentalHealthNotGood
correlated_df = df.copy()
target_var = "Percent_Person_WithMentalHealthNotGood"
noise = np.random.normal(size=(len(correlated_df.index),))
correlated_df["Correlated Variable"] = correlated_df[target_var] + noise

# show new data frame
print("New dataframe to fit:")
display(correlated_df)

# Create a new model
y_corr = correlated_df[dep_var].to_numpy().reshape(-1, 1)
x_corr = correlated_df.loc[:, ~correlated_df.columns.isin([dep_var, "City Name"])]
x_corr = sm.add_constant(x_corr)

correlated_model = sm.OLS(y_corr, x_corr)
correlated_results = correlated_model.fit()

print("Correlated Model Weights and Intercept:")
display(correlated_results.params.round(5))

New dataframe to fit:


,City Name,Count_Person,Percent_Person_Obesity,Percent_Person_PhysicalInactivity,Percent_Person_SleepLessThan7Hours,Percent_Person_WithHighBloodPressure,Percent_Person_WithHighCholesterol,Percent_Person_WithMentalHealthNotGood,Correlated Variable
place,,,,,,,,,
geoId/0107000,Birmingham,200733,40.1,33.9,44.0,44.9,34.6,17.4,16.774543
geoId/0135896,Hoover,92606,28.8,20.2,33.4,32.4,33.2,12.0,13.701440
geoId/0137000,Huntsville,215006,34.7,28.4,38.7,37.4,33.5,15.4,15.562815
geoId/0150000,Mobile,187041,38.0,29.9,41.0,42.2,35.2,16.3,17.077843
geoId/0151000,Montgomery,200603,36.7,30.7,42.6,41.2,34.5,17.0,17.946941
...,...,...,...,...,...,...,...,...,...
geoId/5548000,Madison,269840,24.9,17.1,31.2,22.1,28.2,13.0,12.058370
geoId/5553000,Milwaukee,577222,38.9,28.9,38.8,31.2,29.9,16.6,17.109574
geoId/5566000,Racine,77816,43.2,28.8,36.7,31.6,33.1,15.1,14.787989


Correlated Model Weights and Intercept:


const                                     9.03472
Count_Person                             -0.00000
Percent_Person_PhysicalInactivity         0.41299
Percent_Person_SleepLessThan7Hours       -0.12546
Percent_Person_WithHighBloodPressure      0.47722
Percent_Person_WithHighCholesterol       -0.25037
Percent_Person_WithMentalHealthNotGood    0.81148
Correlated Variable                      -0.10457
dtype: float64

**Q3.2A)** Compare the new weights of the correlated model with the weights of our original model. What happened to the wieghts corresponding to `Percent_Person_WithMentalHealthNotGood`?

**Q3.2B)** Thinking back to your answers for Q3.1C-E, how might correlated variables affect the interpretation of model weights?